### Rotate cifar100 images

In [1]:
import pandas as pd

In [3]:
def rotate_cifar100_images(cifar100_csv_path:str):
    df_cifar100=pd.read_csv(cifar100_csv_path)
    print(df_cifar100)
#     for img_index in range(df_cifar100.shape):

In [4]:
cifar100_csv_path = "C://Users//me//Desktop//Applied Bootkamp//project2//data//CIFAR-100_dataset.csv"

In [ ]:
rotate_cifar100_images(cifar100_csv_path)

In [5]:
df_cifar100=pd.read_csv(cifar100_csv_path)

In [7]:
print(df_cifar100.shape)

(60000, 3073)


In [ ]:
pixels=[p]

In [ ]:
print(df_cifar100[pixels].shape)